# Импорт библиотек

In [31]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st

# Подключени к БД и выгрузка данных

In [6]:
load_dotenv()

True

In [7]:
POSTGRESQL_KEY = os.getenv('POSTGRESQL_KEY')

engine = create_engine(os.getenv('POSTGRESQL_KEY'))
connection = engine.connect()

In [8]:
train = pd.read_sql(
    '''
    SELECT *
    FROM train_data
    ''', 
    con=connection
)

test = pd.read_sql(
    '''
    SELECT *
    FROM test_data
    ''', 
    con=connection
)

In [9]:
connection.close()

In [10]:
spark = SparkSession.builder.master("local").appName("Credit Scoring").getOrCreate()

23/10/27 00:35:29 WARN Utils: Your hostname, MacBook-Air-Vladimir.local resolves to a loopback address: 127.0.0.1; using 192.168.1.37 instead (on interface en0)
23/10/27 00:35:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/27 00:35:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
train, test = spark.createDataFrame(train), spark.createDataFrame(test)

/Users/vladimir/Desktop/GitHub/CreditScroingProject/scoring_venv/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:485: FutureWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if should_localize and is_datetime64tz_dtype(s.dtype) and s.dt.tz is not None:


# Data Overview

In [21]:
train.show(truncate=False)

23/10/27 00:40:10 WARN TaskSetManager: Stage 12 contains a task of very large size (32598 KiB). The maximum recommended task size is 1000 KiB.


+------+-----------+--------+---------------+----+-----------+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+-------------------------------------------------------------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+----------------------+---------------------+-------------------+-----------------------+--------------------------------+------------------+------------+
|ID    |Customer_ID|Month   |Name           |Age |SSN        |Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Type_of_Loan                                                       |Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|Credit_History_Age    |Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|Paymen

23/10/27 00:40:14 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 12 (TID 10): Attempting to kill Python Worker


In [12]:
train.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Customer_ID: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- SSN: string (nullable = true)
 |-- Occupation: string (nullable = true)
 |-- Annual_Income: string (nullable = true)
 |-- Monthly_Inhand_Salary: double (nullable = true)
 |-- Num_Bank_Accounts: long (nullable = true)
 |-- Num_Credit_Card: long (nullable = true)
 |-- Interest_Rate: long (nullable = true)
 |-- Num_of_Loan: string (nullable = true)
 |-- Type_of_Loan: string (nullable = true)
 |-- Delay_from_due_date: long (nullable = true)
 |-- Num_of_Delayed_Payment: string (nullable = true)
 |-- Changed_Credit_Limit: string (nullable = true)
 |-- Num_Credit_Inquiries: double (nullable = true)
 |-- Credit_Mix: string (nullable = true)
 |-- Outstanding_Debt: string (nullable = true)
 |-- Credit_Utilization_Ratio: double (nullable = true)
 |-- Credit_History_Age: string (nullable = true)
 |-- Payme

### В нашем наборе данных преобладают текстовые данные
### Основной задачей будет преобразовать эти данные таким образом, чтобы количество колонок в датасете не стало слишком большим

In [29]:
train.describe().show()

23/10/27 01:02:50 WARN TaskSetManager: Stage 19 contains a task of very large size (32598 KiB). The maximum recommended task size is 1000 KiB.


+-------+-------+-----------+------+------+------------------+-----------+----------+------------------+---------------------+------------------+------------------+------------------+-----------------+--------------------+-------------------+----------------------+--------------------+--------------------+----------+------------------+------------------------+--------------------+---------------------+-------------------+-----------------------+--------------------+--------------------+------------+
|summary|     ID|Customer_ID| Month|  Name|               Age|        SSN|Occupation|     Annual_Income|Monthly_Inhand_Salary| Num_Bank_Accounts|   Num_Credit_Card|     Interest_Rate|      Num_of_Loan|        Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|  Outstanding_Debt|Credit_Utilization_Ratio|  Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|   Payment_Behaviour|     Monthly_Balance|Cred

In [35]:
train.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in train.columns]).show(truncate=False)

23/10/27 01:13:07 WARN TaskSetManager: Stage 25 contains a task of very large size (32598 KiB). The maximum recommended task size is 1000 KiB.


+---+-----------+-----+----+---+---+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+------------+-------------------+----------------------+--------------------+--------------------+----------+----------------+------------------------+------------------+---------------------+-------------------+-----------------------+-----------------+---------------+------------+
|ID |Customer_ID|Month|Name|Age|SSN|Occupation|Annual_Income|Monthly_Inhand_Salary|Num_Bank_Accounts|Num_Credit_Card|Interest_Rate|Num_of_Loan|Type_of_Loan|Delay_from_due_date|Num_of_Delayed_Payment|Changed_Credit_Limit|Num_Credit_Inquiries|Credit_Mix|Outstanding_Debt|Credit_Utilization_Ratio|Credit_History_Age|Payment_of_Min_Amount|Total_EMI_per_month|Amount_invested_monthly|Payment_Behaviour|Monthly_Balance|Credit_Score|
+---+-----------+-----+----+---+---+----------+-------------+---------------------+-----------------+---------------+-------------+-----------+---

### Относительно небольшое количество пропусков